### Pytorch Deep Learning Blitz


In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import operator
from functools import reduce

In [ ]:
x = torch.Tensor(5, 3)
x = torch.randn(5, 3)

In [ ]:
a, b = x.size()
print(a)
print(b)

In [4]:
a = torch.zeros(10)
a_np = a.numpy()
a_np

In [5]:
import numpy as np
a = np.ones(10)
a
b = torch.from_numpy(a)
np.add(a, 10, out = a)
b


 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
[torch.DoubleTensor of size 10]

In [6]:
x = Variable(torch.ones(2, 2), requires_grad = True)

In [7]:
print(x)
y = x + 2
z = y * y * 3
m = z.mean()
print(m.backward())
x.grad

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

None


Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]

In [8]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2, 2))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, (2,2))
        x = x.view(-1, reduce(operator.mul, x.size(), 1))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


In [9]:
params = list(net.parameters())
print(len(params))
params[1].size()

10


torch.Size([6])

In [10]:
input = Variable(torch.randn(1, 1, 32, 32))

output = net(input)
target = Variable(torch.arange(1, 11))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.9447
[torch.FloatTensor of size 1]



In [13]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward(retain_graph = True)

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)
net.zero_grad()
print(net.conv1.bias.grad)
loss.backward(retain_graph = True)
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
 -4.5109
  8.4362
 -2.9122
  5.2665
 -0.5917
 -0.9953
[torch.FloatTensor of size 6]

Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

Variable containing:
1.00000e-02 *
 -4.5109
  8.4362
 -2.9122
  5.2665
 -0.5917
 -0.9953
[torch.FloatTensor of size 6]



In [16]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward() # calculate the gradients
print(optimizer.step())    # Does the update

None


In [17]:
print(target)

Variable containing:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
[torch.FloatTensor of size 10]

